In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import sys
from loguru import logger
import arrow

sys.path.append("..")
from data import get_config

data_path = Path("../data")

logger.remove()
logger.add(sys.stderr, level="INFO")

2024-02-23 10:40:22.436 | INFO     | data:<module>:15 - data_path: g:\Arvin-Edison WSD-1215\121523003-Frick Unit Pipeline\400 GIS\Scripts\aewsd_frick_unit\util\..\data


2

In [2]:
full_config = get_config("Full")
simple_config = get_config("Simple")
full_config.head(2)

,id,clip_to_district,clip_to_frick,Name,shape_type,color,size,alpha,label,file_path,file_type,shp,gdb,parquet,layer,clip_to_frick_x,full,simple
15,5.0,NaN,NaN,FFPPP Discharge Pipeline,line,#A2678A,4.0,1.0,CANAL,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,NaN,NaN,1.0,1.0
14,4.0,NaN,NaN,AEWSD North Canal,line,blue,4.0,1.0,CANAL,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,NaN,NaN,1.0,1.0


In [3]:
district_boundary = gpd.read_file(
    full_config.loc[full_config["Name"] == "District Boundary"]["file_path"].iloc[0]
)
service_boundary = gpd.read_file(
    full_config.loc[full_config["Name"] == "Frick Unit Service Area"]["file_path"].iloc[
        0
    ]
)

intersect = lambda gdf: gdf[gdf.intersects(service_boundary.unary_union)]
clip = lambda gdf: gdf.clip(service_boundary)


def get_layer(row, sheet_name):
    try:
        logger.info(f"Loading layer {row['Name']}")
        file_path = Path(row["file_path"])
        read_options = {
            ".shp": lambda file: gpd.read_file(file),
            ".gdb": lambda file: gpd.read_file(file, layer=row["layer"]),
            ".parquet": lambda file: gpd.read_parquet(file),
        }
        # logger.info(file_path)
        gdf = read_options[file_path.suffix](file_path).to_crs(epsg=2229)

        if row["Name"] == "APNs":
            if sheet_name == "Full":
                gdf["label"] = (
                    gdf["label"] + "\nService Area Type: " + gdf["Service Area Type"]
                )
            gdf["label"] = gdf["label"].str.replace(r"\n", "<br>", regex=True)

            # gdf = full_gdf[full_gdf['layer']==layer]
        if row["Name"] == "Panama Unit Service Area":
            gdf = gdf.loc[gdf["Name"] != "Frick Unit North Service Area"]

        if row["Name"] == "Panama Unit Pipeline":
            gdf = gdf.loc[gdf["Name"] != "Frick Unit"]

        gdf["color"] = row["color"]
        # gdf["label"] = gdf[row["label"]].astype(str)
        # print(gdf.columns)
        if type(row["label"]) == str:
            gdf["label"] = gdf[row["label"]].astype(str).replace("nan", "")
        else:
            gdf["label"] = ""

        gdf["layer"] = row["Name"]
        gdf["size"] = row["size"]
        if row["clip_to_frick"] == True:
            logger.info(f"Clipping layer {row['Name']}")
            gdf = clip(gdf)

        return gdf[
            ["color", "label", "layer", "size", "geometry"]
        ]  # .to_crs(epsg=4326)
    except Exception as e:
        logger.error(f"Layer {row['Name']} failed to load due to {e}")
        logger.error(gdf.columns)
        return None

In [4]:
def get_sheet(sheet_name):
    config = full_config if sheet_name == "Full" else simple_config
    gdfs = {y["Name"]: get_layer(y, sheet_name) for i, y in config.iterrows()}
    epsg = 4326
    gdf = pd.concat([gdf.to_crs(epsg=epsg) for gdf in gdfs.values() if gdf is not None])
    gdf.to_parquet(
        data_path / f"{sheet_name}-{arrow.now().format('YYYY-MM-DD')}.parquet"
    )


get_sheet("Full")
get_sheet("Simple")

2024-02-23 10:40:24.365 | INFO     | __main__:get_layer:16 - Loading layer FFPPP Discharge Pipeline
2024-02-23 10:40:24.402 | INFO     | __main__:get_layer:16 - Loading layer AEWSD North Canal
2024-02-23 10:40:24.441 | INFO     | __main__:get_layer:16 - Loading layer District Boundary
2024-02-23 10:40:24.473 | INFO     | __main__:get_layer:16 - Loading layer Sandrini Unit Service Area
2024-02-23 10:40:24.512 | INFO     | __main__:get_layer:16 - Loading layer Sandrini Unit proposed pipeline
2024-02-23 10:40:24.560 | INFO     | __main__:get_layer:16 - Loading layer Panama Unit Service Area
2024-02-23 10:40:24.596 | INFO     | __main__:get_layer:16 - Loading layer Panama Unit Pipeline
2024-02-23 10:40:24.632 | INFO     | __main__:get_layer:16 - Loading layer Existing Laterals
2024-02-23 10:40:24.674 | INFO     | __main__:get_layer:16 - Loading layer DiGiorgio Service Area
2024-02-23 10:40:24.725 | INFO     | __main__:get_layer:16 - Loading layer DiGiorgio Pipeline
2024-02-23 10:40:24.759 